In [4]:
import argparse
import yaml
from easydict import EasyDict
from diffmot import DiffMOT

with open('configs/dancetrack.yaml') as f:
    config = yaml.safe_load(f)

class Args:
    def __init__(self):
        self.config = 'configs/dancetrack.yaml'
        self.dataset = 'dancetrack'
        self.skip_connection = False
        self.network = 'ReUNet'

    def items(self):
        return vars(self).items()
    
args = Args()

for k, v in args.items():
    config[k] = v

config["exp_name"] = args.config.split("/")[-1].split(".")[0]
config["dataset"] = args.dataset
config = EasyDict(config)

In [5]:
from models.autoencoder import D2MP
from models.condition_embedding import History_motion_embedding

encoder = History_motion_embedding()
model = D2MP(config, encoder=encoder)

AttributeError: 'EasyDict' object has no attribute 'add_linear'